In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
import seaborn as sns
train=pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
train.head()
train.describe()
train.isnull().sum()

In [ ]:
corr=train.corr()
corr.style.background_gradient(cmap='coolwarm') 

In [ ]:
train['Pawpularity'].hist()

In [ ]:
classifiers=[KNeighborsRegressor(5),
            SVC(probability=True,kernel='linear'),
            RandomForestRegressor(n_estimators=200,max_depth=5,random_state=612),
            GradientBoostingRegressor(n_estimators=200,max_depth=5,random_state=612)]
rmse_dict={}
log_cols=['Algoritm','RMSE']
log=pd.DataFrame(columns=log_cols)
test=pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
X_train, X_test, Y_train, Y_test = train_test_split(train.drop(['Id', 'Pawpularity'], axis=1), 
                                                    train['Pawpularity'], 
                                                    test_size=0.2, 
                                                    random_state=612)
for clf in classifiers:
    name=clf.__class__.__name__
    clf.fit(X_train,Y_train)
    predictions=clf.predict(X_test)
    rmse=mean_squared_error(Y_test,predictions,squared=False)
    print(clf,rmse)
    if name in rmse_dict:
        rmse_dict[name]+=rmse
    else:
        rmse_dict[name]=rmse
for clf in rmse_dict:
    log_entry=pd.DataFrame([[clf,rmse_dict[clf]]],columns=log_cols)
    log=log.append(log_entry)
plt.xlabel('RMSE') 
import seaborn as sns
sns.set_color_codes('muted')
sns.barplot(x='RMSE',y='Algoritm',data=log.sort_values(by='RMSE'),color='b')

the best model is RandomForestRegressor, rmse=20,716

In [ ]:
model=RandomForestRegressor(n_estimators=200,max_depth=5,random_state=612)
model_fit=model.fit(X_train,Y_train)
pred=model_fit.predict(X_test)
rmse=mean_squared_error(Y_test,pred,squared=False)
print(rmse)

In [ ]:
sub = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")
sub['Pawpularity'] = model_fit.predict(test.drop(['Id'], axis=1))
sub.to_csv("submission.csv", index=False)
sub